In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 🔹 Función para buscar archivos en la jerarquía de carpetas
def find_file(filename, search_dir=None):
    """
    Busca un archivo en la jerarquía de carpetas hacia arriba hasta encontrarlo.
    Si search_dir no se especifica, comienza desde el directorio actual.
    """
    search_dir = Path(search_dir) if search_dir else Path.cwd()
    
    # Buscar en la carpeta actual y subir niveles hasta encontrar el archivo
    for parent in [search_dir] + list(search_dir.parents):
        possible_path = parent / "data" / filename
        if possible_path.exists():
            return possible_path.resolve()  # Retorna la ruta absoluta

    raise FileNotFoundError(f"❌ No se encontró el archivo {filename} en la estructura de directorios.")

# 🔹 Función de ingeniería de características
def feature_engineering(df, is_test=False):
    """Genera características adicionales para el modelo."""
    
    # Extraer día de la semana y hora del viaje
    df["day_of_week"] = df["start_time"].dt.dayofweek  # Lunes=0, Domingo=6
    df["hour"] = df["start_time"].dt.hour  # Hora del día
    
    # Marcar si es fin de semana
    df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)
    
    # Categorizar duración del viaje
    df["duration_category"] = pd.cut(df["duration"],
                                     bins=[0, 5, 15, 30, 60, 120, 360],
                                     labels=["0-5 min", "5-15 min", "15-30 min", "30-60 min", "60-120 min", "120+ min"])
    
    # Eliminar columnas irrelevantes solo si existen
    cols_to_drop = ["bike_id", "start_time", "end_time"]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

    # Convertir variables categóricas a numéricas
    categorical_cols = ["trip_route_category", "start_station", "end_station", "duration_category"]

    if not is_test:
        categorical_cols.append("passholder_type")  # Solo en entrenamiento

    label_encoders = {}  # Para almacenar los codificadores de etiquetas

    for col in categorical_cols:
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col].astype(str))
        label_encoders[col] = encoder  # Guardamos los encoders para después

    return df, label_encoders

# 🔹 Procesar el dataset de entrenamiento
try:
    df_clean_path = find_file("df_clean.feather")
    print(f"✅ Archivo encontrado en: {df_clean_path}")
    df_clean = pd.read_feather(df_clean_path)
except FileNotFoundError as e:
    print(e)
    exit(1)

df_features, label_encoders = feature_engineering(df_clean)

# Separar variables predictoras (X) y variable objetivo (y)
X = df_features.drop(columns=["passholder_type"])
y = df_features["passholder_type"]

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Buscar la ruta correcta para guardar los archivos
train_data_path = find_file("data") / "train.feather"
test_data_path = find_file("data") / "test.feather"

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_feather(train_data_path)
test_data.to_feather(test_data_path)

print(f"✅ Feature engineering completado. Archivos guardados en:\n - {train_data_path}\n - {test_data_path}")

# 🔹 Procesar el dataset de prueba (`df_clean_test.feather`)
try:
    df_test_path = find_file("df_clean_test.feather")
    print(f"✅ Archivo de test encontrado en: {df_test_path}")
    df_test = pd.read_feather(df_test_path)
except FileNotFoundError as e:
    print(e)
    exit(1)

df_test_features, _ = feature_engineering(df_test, is_test=True)  # No necesita `passholder_type`

# Guardar dataset de prueba
test_processed_path = find_file("data") / "test_processed.feather"
df_test_features.to_feather(test_processed_path)

print(f"✅ Dataset de prueba procesado. Archivo guardado en: {test_processed_path}")



✅ Archivo encontrado en: C:\Users\candy\Downloads\arkon_ds\data\df_clean.feather


FileNotFoundError: ❌ No se encontró el archivo data en la estructura de directorios.

In [5]:
from pathlib import Path

file_path = Path("data/df_clean.feather")
print("¿El archivo existe?", file_path.exists())
print("Ruta absoluta:", file_path.resolve())


¿El archivo existe? False
Ruta absoluta: C:\Users\candy\Downloads\arkon_ds\notebooks\data\df_clean.feather
